# Calculating population density

We want to know how many individuals are exposed to flares at each of the study regions.

To do this we want to:
- read in all population data from the study regions
- calculate the total population density within each of the study regions
-  using literatiure select an appropriate threshhold for AOD to use as a cutoff point for the "high levels of pollution"
- from this calculate total no. of people and % of people in area affected by high levels of pollution

###  Literature
Taking a look at this articles:
- [What Can Affect AOD–PM2.5 Association?](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2854780/)
- [An empirical relationship between PM2.5 and aerosol optical depth in Delhi Metropolitan](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3237057/)
  - "1% change in AOD explains 0.52±0.202% and 0.39±0.15% change in PM2.5 monitored within ±45 and 150 min intervals of AOD data."
    - [Table with PM2.5 to AOD conversions (for Dehli](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3237057/table/T1/)
      - PM2.5 (μg m−3) 30.5 = AOD (5 km) 172.1 

In [24]:
import os

# Reading in all population data
import geopandas as gpd
base_dir = os.path.dirname(os.getcwd())

In [54]:
gwer_road = gpd.read_file(f"{base_dir}/pop_dens_data/polygon_geojson/POP_DEN_gwer_polygon.geojson")
kalak = gpd.read_file(f"{base_dir}/pop_dens_data/polygon_geojson/POP_DEN_kalak_polygon.geojson")
lalish = gpd.read_file(f"{base_dir}/pop_dens_data/polygon_geojson/POP_DEN_lalish_polygon.geojson")

In [55]:
gwer_road.head()

,VALUE,geometry
0,441.916229,"MULTIPOLYGON (((43.78208 36.24139, 43.78208 36..."
1,419.327271,"MULTIPOLYGON (((43.78547 36.24193, 43.78977 36..."
2,511.886505,"MULTIPOLYGON (((43.74875 36.23515, 43.74875 36..."
3,441.208771,"MULTIPOLYGON (((43.75148 36.23655, 43.75708 36..."
4,245.390091,"MULTIPOLYGON (((43.76542 36.23875, 43.76542 36..."


In [56]:
sum(gwer_road["VALUE"])

121183.42779943

In [57]:
sum(gwer_road["VALUE"])

121183.42779943

In [58]:
sum(kalak["VALUE"])

219058.2630378402

In [59]:
sum(lalish["VALUE"])

40925.76306725002

Next we want to read in the `AOD` data which quanitifies atmospheric haziness and gives us an idea of the level of pollution in the atmosphere.


In [60]:
gwer_road_aod = gpd.read_file(f"{base_dir}/aod_data/aod_polygons/AOD_gwer_polygon.geojson")
kalak_aod = gpd.read_file(f"{base_dir}/aod_data/aod_polygons/AOD_kalak_polygon.geojson")
lalish_aod = gpd.read_file(f"{base_dir}/aod_data/aod_polygons/AOD_lalish_polygon.geojson")

In order to understand the exposure to `AOD` we need to count the population exposed to `AOD` concentrations above an average of the following threshhold:
- (170.2/30.5 μg/m3 )*5 μg/m3  = 27 μg/m3  [source here](https://www.ersnet.org/wp-content/uploads/2021/10/WHO-AQG_Joint-Society-Statement_1st-UPDATE-13th-October.pdf)

In [71]:
gwer_road_mean_aod = gwer_road_aod.groupby(["longitude", "latitude", "VALUE"])["Optical_Depth_047"].mean().reset_index()

kalak_mean_aod = kalak_aod.groupby(["longitude", "latitude", "VALUE"])["Optical_Depth_047"].mean().reset_index()

lalish_mean_aod = lalish_aod.groupby(["longitude", "latitude", "VALUE"])["Optical_Depth_047"].mean().reset_index()

Calculate the percentage of population exposed to rates of PM2.5 (using the AOD-> PM2.5 conversion) exceeding the WHO annual mean concentrations recommendations.

In [65]:
gwer_road_harmful_aod = gwer_road_mean_aod[gwer_road_mean_aod["Optical_Depth_047"]>175]
kalak_harmful_aod = kalak_mean_aod[kalak_mean_aod["Optical_Depth_047"]>175]
lalish_harmful_aod = lalish_mean_aod[lalish_mean_aod ["Optical_Depth_047"]>175]

In [82]:
sum(gwer_road_harmful_aod["VALUE"])/sum(gwer_road_mean_aod['VALUE'])

1.0

In [83]:
sum(kalak_harmful_aod["VALUE"])/sum(kalak_mean_aod['VALUE'])

1.0

In [81]:
sum(lalish_harmful_aod["VALUE"])/sum(lalish_mean_aod['VALUE'])

0.9951336173308959

Exposure rates as follow:
- 100% of kalak population above WHO recommended levels of annual mean concentrations of PM2.5
- 100% of gwer road population above WHO recommended levels of annual mean concentrations of PM2.5
- 99.5% of lalish population above WHO recommended levels of annual mean concentrations of PM2.5

Calculating overall average to get figure for no. of times greater than the WHO recommended limit is for each study region, assuming that `(170.2 AOD / 30.5 μg/m3 ) * 5 μg/m3 (WHO limit) = 27 AOD`

In [93]:
gwer_road_aod["Optical_Depth_047"].mean()/27

7.154448667905098

In [94]:
kalak_aod["Optical_Depth_047"].mean()/27

7.161162383289336

In [95]:
lalish_aod["Optical_Depth_047"].mean()/27

7.557843945296876

Calculating overall average to get figure for no. of times greater than the WHO recommended limit is for each study region, assuming that `(170.2 AOD / 30.5 μg/m3 ) * 5 μg/m3 (WHO limit) = 27 AOD`
- gwer_road = 7.15 x WHO Limit
- Kalak = 7.16 x WHO Limit
- lalish = 7.56 x WHO Limit